# <center>Email Spam Detection </center>

In [43]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
import os

In [4]:
df = pd.read_csv("C:/Users/DELL/Downloads/mail_data.csv")

In [5]:
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [6]:
df.size

11144

In [7]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df.describe

<bound method NDFrame.describe of      Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]>

In [9]:
df.describe()

,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [13]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [17]:
data = df.where((pd.notnull(df)), '')

In [18]:
data.loc[data['Category'] == 'spam', 'Category',] = 0
data.loc[data['Category'] == 'ham', 'Category',] = 1

In [19]:
x = data['Message']
y = data['Category']

In [20]:
print(x)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object


In [21]:
print(y)

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object


## Splitting the dataset

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x,y ,test_size = 0.2, random_state =3)

In [23]:
x.shape, x_train.shape, x_test.shape,y.shape, y_train.shape, y_test.shape

((5572,), (4457,), (1115,), (5572,), (4457,), (1115,))

In [24]:
x_train.head()

3075                  Don know. I did't msg him recently.
1787    Do you know why god created gap between your f...
1614                         Thnx dude. u guys out 2nite?
4304                                      Yup i'm free...
3266    44 7732584351, Do you want a New Nokia 3510i c...
Name: Message, dtype: object

In [25]:
y_train.head()

3075    1
1787    1
1614    1
4304    1
3266    0
Name: Category, dtype: object

In [26]:
x_test.head()

2632    URGENT! Your mobile No 077xxx WON a £2,000 Bon...
454     Ok i will tell her to stay out. Yeah its been ...
983     Congrats! 2 mobile 3G Videophones R yours. cal...
1282        Am I the only one who doesn't stalk profiles?
4610                               Y de asking like this.
Name: Message, dtype: object

In [27]:
y_test.head()

2632    0
454     1
983     0
1282    1
4610    1
Name: Category, dtype: object

## Feature Extraction

In [28]:
feature_extraction = TfidfVectorizer(min_df = 1, stop_words = 'english', lowercase = True)
x_train_features = feature_extraction.fit_transform(x_train)
x_test_features = feature_extraction.transform(x_test)
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [29]:
print(x_train_features)

  (0, 5413)	0.6198254967574347
  (0, 4456)	0.4168658090846482
  (0, 2224)	0.413103377943378
  (0, 3811)	0.34780165336891333
  (0, 2329)	0.38783870336935383
  (1, 4080)	0.18880584110891163
  (1, 3185)	0.29694482957694585
  (1, 3325)	0.31610586766078863
  (1, 2957)	0.3398297002864083
  (1, 2746)	0.3398297002864083
  (1, 918)	0.22871581159877646
  (1, 1839)	0.2784903590561455
  (1, 2758)	0.3226407885943799
  (1, 2956)	0.33036995955537024
  (1, 1991)	0.33036995955537024
  (1, 3046)	0.2503712792613518
  (1, 3811)	0.17419952275504033
  (2, 407)	0.509272536051008
  (2, 3156)	0.4107239318312698
  (2, 2404)	0.45287711070606745
  (2, 6601)	0.6056811524587518
  (3, 2870)	0.5864269879324768
  (3, 7414)	0.8100020912469564
  (4, 50)	0.23633754072626942
  (4, 5497)	0.15743785051118356
  :	:
  (4454, 4602)	0.2669765732445391
  (4454, 3142)	0.32014451677763156
  (4455, 2247)	0.37052851863170466
  (4455, 2469)	0.35441545511837946
  (4455, 5646)	0.33545678464631296
  (4455, 6810)	0.29731757715898277
  (4

In [30]:
print(x_test_features)

  (0, 7271)	0.1940327008179069
  (0, 6920)	0.20571591693537986
  (0, 5373)	0.2365698724638063
  (0, 5213)	0.1988547357502182
  (0, 4386)	0.18353336340308998
  (0, 1549)	0.2646498848307188
  (0, 1405)	0.3176863938914351
  (0, 1361)	0.25132445289897426
  (0, 1082)	0.2451068436245027
  (0, 1041)	0.28016206931555726
  (0, 405)	0.2381316303003606
  (0, 306)	0.23975986557206702
  (0, 20)	0.30668032384591537
  (0, 14)	0.26797874471323896
  (0, 9)	0.2852706805264544
  (0, 1)	0.2381316303003606
  (1, 7368)	0.29957800964520975
  (1, 6732)	0.42473488678029325
  (1, 6588)	0.3298937975962767
  (1, 6507)	0.26731535902873493
  (1, 6214)	0.3621564482127515
  (1, 4729)	0.22965776503163893
  (1, 4418)	0.3457696891316818
  (1, 3491)	0.496093956101028
  (2, 7205)	0.22341717215670331
  :	:
  (1110, 3167)	0.5718357066163949
  (1111, 7353)	0.4991205841293424
  (1111, 6787)	0.40050175714278885
  (1111, 6033)	0.4714849709283488
  (1111, 3227)	0.44384935772735523
  (1111, 2440)	0.4137350055985486
  (1112, 7071)

## Training the model

### Logistic Regression

In [31]:
model = LogisticRegression()
model.fit(x_train_features, y_train )
pred_train_data = model.predict(x_train_features)

In [32]:
accuracy_pred_train_data = accuracy_score(y_train, pred_train_data)
print('Accuracy on Training Data: ', accuracy_pred_train_data)

Accuracy on Training Data:  0.9670181736594121


In [33]:
input_email = ["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's  "]

input_data_features = feature_extraction.transform(input_email)

pred = model.predict(input_data_features)

if(pred == 1):
    print('Not a Spam Mail!')
else:
    print('Spam Mail!')

Spam Mail!


### Support Vector Classifier

In [37]:
model = SVC()
model.fit(x_train_features, y_train )
pred_train_data = model.predict(x_train_features)

In [38]:
accuracy_pred_train_data = accuracy_score(y_train, pred_train_data)
print('Accuracy on Training Data: ', accuracy_pred_train_data)

Accuracy on Training Data:  0.99798070450976


In [39]:
input_email = ["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's  "]

input_data_features = feature_extraction.transform(input_email)

pred = model.predict(input_data_features)

if(pred == 1):
    print('Not a Spam Mail!')
else:
    print('Spam Mail!')

Spam Mail!


### XG Boost Classifier

In [44]:
model = XGBClassifier()
model.fit(x_train_features, y_train )
pred_train_data = model.predict(x_train_features)

In [45]:
accuracy_pred_train_data = accuracy_score(y_train, pred_train_data)
print('Accuracy on Training Data: ', accuracy_pred_train_data)

Accuracy on Training Data:  0.9910253533767108


In [46]:
input_email = ["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's  "]

input_data_features = feature_extraction.transform(input_email)

pred = model.predict(input_data_features)

if(pred == 1):
    print('Not a Spam Mail!')
else:
    print('Spam Mail!')

Spam Mail!


### AdaBoost Classifier

In [47]:
model = AdaBoostClassifier()
model.fit(x_train_features, y_train )
pred_train_data = model.predict(x_train_features)

In [48]:
accuracy_pred_train_data = accuracy_score(y_train, pred_train_data)
print('Accuracy on Training Data: ', accuracy_pred_train_data)

Accuracy on Training Data:  0.9771146511106125


In [49]:
input_email = ["Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's  "]

input_data_features = feature_extraction.transform(input_email)

pred = model.predict(input_data_features)

if(pred == 1):
    print('Not a Spam Mail!')
else:
    print('Spam Mail!')

Spam Mail!
